Q3. Implement matrix multiplication using CUDA C programming. (5 points)
- Use blocks and threads for parallel computation, clearly explain the blocks and threads in the code.
- Profile the CUDA program using nvprof and nsys(Modern NVIDIA GPUs (CUDA 11 and newer))

In [ ]:
!pip install nvcc4jupyter

In [ ]:
%load_ext nvcc4jupyter

The nvcc4jupyter extension is already loaded. To reload it, use:
  %reload_ext nvcc4jupyter


In [ ]:
%%writefile my_cuda_code.cu
#include <stdio.h>
#include <stdlib.h>

// Defining the dimension of square matrices we will use in our code
#define N 512  // Matrix size: N x N


// CUDA Kernel for Matrix Multiplication
// Each GPU thread computes one element of the output matrix C

__global__ void matrixMulCUDA(float *A, float *B, float *C, int width) {
    // Calculate the row index of the element this thread will compute
    int row = blockIdx.y * blockDim.y + threadIdx.y;

    // Calculate the column index of the element this thread will compute
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    // Check if the thread is within matrix bounds
    if (row < width && col < width) {
        float sum = 0.0f;

        // Compute dot product of the corresponding row of A and column of B
        for (int k = 0; k < width; k++) {
            sum += A[row * width + k] * B[k * width + col];
        }

        // Store the computed value in the output matrix C
        C[row * width + col] = sum;
    }
}

int main() {
    int size = N * N * sizeof(float);  // Total size in bytes for each matrix


    // Allocate host memory (CPU)

    float *h_A = (float *)malloc(size);  // Host matrix A
    float *h_B = (float *)malloc(size);  // Host matrix B
    float *h_C = (float *)malloc(size);  // Host result matrix C

    // Initializing host matrices with sample values
    // A filled with 1.0, B filled with 2.0 for simplicity and chekcing the output

    for (int i = 0; i < N * N; i++) {
        h_A[i] = 1.0f;
        h_B[i] = 2.0f;
    }


    // Allocate device memory (GPU)

    float *d_A, *d_B, *d_C;
    cudaMalloc((void **)&d_A, size);  // Device matrix A
    cudaMalloc((void **)&d_B, size);  // Device matrix B
    cudaMalloc((void **)&d_C, size);  // Device result matrix C

    // Copy input data from host (CPU) to device (GPU)

    cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);


    // Defining thread and block configuration
    // Each block contains 16x16 threads = 256 threads per block

    dim3 threadsPerBlock(16, 16);

    // Grid size calculated to cover full N x N matrix
    // Each block handles 16 rows × 16 cols, so we need N/16 blocks per dimension
    // (N + 15)/16 ensures we round up for any leftover rows/cols

    dim3 blocksPerGrid((N + 15) / 16, (N + 15) / 16);

    // Launch the kernel on the GPU

    matrixMulCUDA<<<blocksPerGrid, threadsPerBlock>>>(d_A, d_B, d_C, N);

    // Wait for kernel to complete before copying result back
    cudaDeviceSynchronize();

    // Check for any kernel launch errors
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        printf("CUDA Error: %s\n", cudaGetErrorString(err));
        return 1;
    }

    // Copy the result matrix from device back to host
    cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost);


    // Output
    // Print the first element as a basic check
    // Expected value: 512 * (1.0 * 2.0) = 1024.0

    printf("C[0] = %f\n", h_C[0]);

    // Free allocated memory

    cudaFree(d_A); cudaFree(d_B); cudaFree(d_C);
    free(h_A); free(h_B); free(h_C);

    return 0;
}


Overwriting my_cuda_code.cu


In the above CUDA matrix multiplication program, we use a 2D grid of 2D thread blocks to parallelize the computation. Each block has 16×16 = 256 threads, and the grid is sized so that all elements of a 512×512 matrix are covered. This results in a 32×32 grid of blocks, giving us a total of 1024 blocks and 262,144 threads—one for each element in the output matrix. Inside the kernel, each thread figures out its row and column position in the matrix using the formulas row = blockIdx.y * blockDim.y + threadIdx.y and col = blockIdx.x * blockDim.x + threadIdx.x. This tells each thread exactly which element of the matrix it's responsible for computing. Each thread then computes the value for one element in the result matrix by taking the dot product of a row from matrix A and a column from matrix B.

In [ ]:
!nvcc -arch=sm_75 -o my_cuda_code my_cuda_code.cu


In [ ]:
!./my_cuda_code

C[0] = 1024.000000


In [ ]:
!nvprof ./my_cuda_code

==27551== NVPROF is profiling process 27551, command: ./my_cuda_code
C[0] = 1024.000000
==27551== Profiling application: ./my_cuda_code
==27551== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   80.64%  1.1472ms         1  1.1472ms  1.1472ms  1.1472ms  matrixMulCUDA(float*, float*, float*, int)
                   12.29%  174.91us         2  87.455us  87.391us  87.519us  [CUDA memcpy HtoD]
                    7.07%  100.51us         1  100.51us  100.51us  100.51us  [CUDA memcpy DtoH]
      API calls:   98.30%  176.42ms         3  58.807ms  3.0820us  176.35ms  cudaMalloc
                    0.77%  1.3737ms         3  457.89us  239.71us  821.57us  cudaMemcpy
                    0.64%  1.1516ms         1  1.1516ms  1.1516ms  1.1516ms  cudaDeviceSynchronize
                    0.13%  230.64us         3  76.880us  14.890us  114.43us  cudaFree
                    0.08%  137.23us         1  137.23us  137.23us  137.23us  cuda

In [ ]:
!nsys profile --stats=true ./my_cuda_code


/bin/bash: line 1: nsys: command not found


Q4. Implement matrix addition (C = A + B) for 1024×1024 matrices using CUDA C with 2D thread blocks (16×16 threads per block). In your code comments, explain how 2D thread indexing works and why it's more intuitive for matrix operations than 1D configuration. (5 points)

In [ ]:
%%writefile matrix_add.cu
#include <stdio.h>
#include <stdlib.h>

#define N 1024  // Define the size of the square matrix: 1024 x 1024

// CUDA Kernel for Matrix Addition

__global__ void matrixAdd(float *A, float *B, float *C, int width) {

    // 2D THREAD INDEXING EXPLANATION

    // Each thread is assigned to compute one unique element in the output matrix C.
    // To find the position of the element this thread is responsible for,
    // we compute its row and column based on:
    //
    // - blockIdx: index of the block within the grid (x = columns, y = rows)
    // - blockDim: number of threads per block (e.g., 16x16 = 256 threads)
    // - threadIdx: index of the thread within its own block
    //
    // This results in the global row and column index of the thread:
    int row = blockIdx.y * blockDim.y + threadIdx.y; // Row index for the thread
    int col = blockIdx.x * blockDim.x + threadIdx.x; // Column index for the thread

    // Convert the (row, col) 2D position into a 1D index for linear memory access
    int index = row * width + col;

    // Check if the thread is within matrix bounds (for edge cases)
    if (row < width && col < width) {
        // Each thread adds corresponding elements from A and B
        C[index] = A[index] + B[index];
    }

    // 2D thread indexing is more intuitive for matrix operations because:
    // - It directly maps threads to matrix elements using (row, col) coordinates
    // - This matches how we naturally access matrices in C: C[row][col]
    // - It avoids the need to manually convert from 1D thread IDs using
    //   division (%) and multiplication, making the code easier to read and maintain
    // - It's especially helpful for large matrices where visualizing and debugging
    //   becomes more complex with 1D indexing
}

int main() {
    int size = N * N * sizeof(float);  // Total size of each matrix in bytes

    // =======================
    // Host Memory Allocation
    // =======================
    float *h_A = (float *)malloc(size);  // Matrix A on host
    float *h_B = (float *)malloc(size);  // Matrix B on host
    float *h_C = (float *)malloc(size);  // Result matrix C on host

    // Initialize host matrices
    for (int i = 0; i < N * N; i++) {
        h_A[i] = 1.0f;  // Fill matrix A with 1.0
        h_B[i] = 2.0f;  // Fill matrix B with 2.0
    }

    // =======================
    // Device Memory Allocation
    // =======================
    float *d_A, *d_B, *d_C;
    cudaMalloc((void **)&d_A, size);  // Allocate matrix A on device
    cudaMalloc((void **)&d_B, size);  // Allocate matrix B on device
    cudaMalloc((void **)&d_C, size);  // Allocate result matrix C on device

    // Copy input matrices from host to device
    cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);

    // =======================
    // Thread & Block Configuration
    // =======================
    // Each block has 16 x 16 = 256 threads (2D configuration)
    dim3 threadsPerBlock(16, 16);

    // Grid size is calculated to ensure full matrix coverage
    // (N + 15) / 16 ensures we round up to cover all rows/columns
    dim3 blocksPerGrid((N + 15) / 16, (N + 15) / 16);

    // Launch the CUDA kernel
    matrixAdd<<<blocksPerGrid, threadsPerBlock>>>(d_A, d_B, d_C, N);

    // Wait for the kernel to complete
    cudaDeviceSynchronize();

    // Copy the result matrix from device back to host
    cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost);

    // Print a sample output to verify if the code is giving ecpected output of 3
    printf("C[0] = %f ", h_C[0]);

    // Free all allocated memory
    cudaFree(d_A); cudaFree(d_B); cudaFree(d_C);
    free(h_A); free(h_B); free(h_C);

    return 0;
}

Overwriting matrix_add.cu


In [ ]:
!nvcc -arch=sm_75 -o matrix_add matrix_add.cu


In [ ]:
!./matrix_add


C[0] = 3.000000 

In [ ]:
!nvprof ./matrix_add

==27676== NVPROF is profiling process 27676, command: ./matrix_add
C[0] = 3.000000 ==27676== Profiling application: ./matrix_add
==27676== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   50.71%  1.6457ms         1  1.6457ms  1.6457ms  1.6457ms  [CUDA memcpy DtoH]
                   47.54%  1.5427ms         2  771.37us  765.01us  777.72us  [CUDA memcpy HtoD]
                    1.75%  56.671us         1  56.671us  56.671us  56.671us  matrixAdd(float*, float*, float*, int)
      API calls:   96.92%  179.30ms         3  59.767ms  70.527us  179.16ms  cudaMalloc
                    2.62%  4.8450ms         3  1.6150ms  937.61us  2.9229ms  cudaMemcpy
                    0.27%  506.67us         3  168.89us  108.28us  199.28us  cudaFree
                    0.07%  133.38us         1  133.38us  133.38us  133.38us  cudaLaunchKernel
                    0.07%  132.52us       114  1.1620us     105ns  54.285us  cuDeviceGetAttribute

In [ ]:
!nsys profile --stats=true ./matrix_add

/bin/bash: line 1: nsys: command not found
